# Pandas with SQLite 

SQLite is a database engine that makes it simple to store and work with relational data. Most programming languages have good support for working with SQLite databases. Python is too. In this section, I will create table with pandas.

In [1]:
%ls

cheat_sheet_part1.ipynb             cheat_sheet_part5.ipynb
cheat_sheet_part2.ipynb             converting_date_numeric_par1.ipynb
cheat_sheet_part3.ipynb             flights.db
cheat_sheet_part4.ipynb             pd_sqlite.ipynb


In [2]:
# Connect with Sqlite
import sqlite3
conn = sqlite3.connect('flights.db')

In [3]:
# Create Cursor object. Cursors allow us to excute SQL queries
cur = conn.cursor()
cur.execute('select * from airlines limit 5;')

In [4]:
# Using fetchall to display table
results = cur.fetchall()
print(results)

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'), (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'), (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'), (3, '4', '2 Sqn No 1 Elementary Flying Training School', '\\N', None, 'WYT', None, 'United Kingdom', 'N'), (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]


This resuts are formatted as a list of tuples. Each tuple corresponds to a row in the database that we accessed. We need to add column head, and parse the data.

In [5]:
# Before move on to Mapping we have to close Connection object and cursor
cur.close()
conn.close()

## Read db

In [6]:
# I query lat and log in airports data
import sqlite3

conn = sqlite3.connect('flights.db')
cur = conn.cursor()

# query statement
coords = cur.execute("""
    select cast (longitude as float),
    cast (latitude as float)
    from airports;
""")

coResult = coords.fetchall()

In [7]:
conn.close()

## Read db into Pandas

In [8]:
import pandas as pd

conn = sqlite3.connect('flights.db')
df = pd.read_sql_query('select * from airlines limit 5;', conn)
df

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [9]:
df['country']

0              None
1     United States
2      South Africa
3    United Kingdom
4            Russia
Name: country, dtype: object

### * Insert rows

In [10]:
cur  = conn.cursor()
cur.execute("insert into airlines values (6048, 19846, 'Test flight', '', '', null, null, null, 'Y' )")

In [11]:
%ls

cheat_sheet_part1.ipynb             converting_date_numeric_par1.ipynb
cheat_sheet_part2.ipynb             flights.db
cheat_sheet_part3.ipynb             flights.db-journal
cheat_sheet_part4.ipynb             pd_sqlite.ipynb
cheat_sheet_part5.ipynb


flights.db-journal. flights.db-journal are created and storing the new row until I ready to commit it to the main database.

#### Transaction

SQLite3 does not write to the database until I commit a transaction. A transaction consists of 1 or more queries and run several queries, then finally alter the database with the results of all of them. If one of the queries fails, the database isn’t partially updated.

In [12]:
conn.commit()

df = pd.read_sql_query("select * from airlines", conn)
print(df.shape)

(6049, 9)


In [13]:
df.tail()

,index,id,name,alias,iata,icao,callsign,country,active
6044,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y
6045,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y
6046,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y
6047,6047,19845,FTI Fluggesellschaft,None,None,FTI,None,Germany,N
6048,6048,19846,Test flight,,,None,None,None,Y


In [14]:
# or Use where statement
pd.read_sql_query("select * from airlines where id = 19846", conn)

,index,id,name,alias,iata,icao,callsign,country,active
0,6048,19846,Test flight,,,None,None,None,Y


### * Passing Parameter into a query

Most of the time, when we insert data into a database, it won’t be hardcoded, it will be dynamic values we want to pass in. These dynamic values might come from downloaded data, or come from user. When working with dynamic data, it might be tempting to insert values using Python string formatting like below.

In [15]:
#name = "Test2 Flight"
#cur.execute("insert into airlines values (6049, 19847, {0}, ' ', ' ', null, null, null, 'Y')").format(name)
#conn.commit()

Sqlite3 has a straightforward way to inject dynamic values without relying on string formatting.

In [16]:
value = ('Test2 flight', 'Y')
cur.execute("insert into airlines values (6049, 19847, ?, ' ' , ' ' , null, null, null, ?)", value)
conn.commit()

In [17]:
df = pd.read_sql_query("select * from airlines", conn)
df.tail()

,index,id,name,alias,iata,icao,callsign,country,active
6045,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y
6046,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y
6047,6047,19845,FTI Fluggesellschaft,None,None,FTI,None,Germany,N
6048,6048,19846,Test flight,,,None,None,None,Y
6049,6049,19847,Test2 flight,,,None,None,None,Y


### * Updating rows

In [18]:
values = ('USA', 19847)
cur.execute("update airlines set country = ? where id =?", values)
conn.commit()

In [19]:
pd.read_sql_query("select * from airlines where id=19847", conn)

,index,id,name,alias,iata,icao,callsign,country,active
0,6049,19847,Test2 flight,,,None,None,USA,Y


### * Deleting rows

In [20]:
values = (19847, )
cur.execute("delete from airlines where id=?", values)
conn.commit()

In [21]:
pd.read_sql_query("select * from airlines where id = 19847", conn)

,index,id,name,alias,iata,icao,callsign,country,active


### * Creating tables

In [22]:
cur.execute("create table daily_flights( id integer, departure date, arrival date, number text, route_id integer)")

conn.commit()

In [23]:
pd.read_sql_query("select * from daily_flights", conn)

,id,departure,arrival,number,route_id


In [24]:
cur.execute("insert into daily_flights values(1, '2016-09-28 0:00', '2016-09-28 12:00', 'T1', 1)")
conn.commit()

In [25]:
pd.read_sql_query("select * from daily_flights", conn)

,id,departure,arrival,number,route_id
0,1,2016-09-28 0:00,2016-09-28 12:00,T1,1


### * Creating tables with Pandas

In [26]:
from datetime import datetime

df = pd.DataFrame(
    [[1, datetime(2016, 9, 29, 0, 0), datetime(2016, 9, 29, 12, 0), 'T2', 2]
    ],
    columns=['id', 'departure', 'arrival', 'number', 'route_id']
)

df

,id,departure,arrival,number,route_id
0,1,2016-09-29,2016-09-29 12:00:00,T2,2


In [27]:
df.to_sql("daily_filghts", conn, if_exists="replace")

In [28]:
pd.read_sql_query("select * from daily_flights;", conn)

,id,departure,arrival,number,route_id
0,1,2016-09-28 0:00,2016-09-28 12:00,T1,1


### * Altering tables with Pandas

In [29]:
cur.execute("alter table airlines add column airplanes int;")

In [30]:
pd.read_sql_query("select * from airlines  limit 5", conn)

,index,id,name,alias,iata,icao,callsign,country,active,airplanes
0,0,1,Private flight,\N,-,None,None,None,Y,None
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N,None
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y,None
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N,None
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N,None
